In [8]:
import sys
import os
from dash import Dash, dcc, html, Input, Output, callback, dash_table
import dash
import plotly.express as px
import base64
import pandas as pd
import dash_leaflet as dl
import numpy as np
import matplotlib.pyplot as plt
from CRUD_Module_main.animal_shelter import animalShelter
external_stylesheets = ['assets/styles.css']

module_path = os.path.dirname(os.path.abspath('CRUD-Module-main.animal_shelter'))
sys.path.insert(0, module_path)


username = "aacuser"
password = "SNHU1234"
collection = 'animals'
database = "aac"
port = 8050
host = '127.0.0.1'
df = pd.read_csv('Austin-Animal-Outcomes-CSV/aac_shelter_outcomes.csv')


#Colors for the layout - 
#171a1e (darker grey) background color of the webpage
#d2d2d2 (white) dashboard title, top level fields
#d24b1a (red) - logo
#171a1e (grey) - background color of dash
#53e51d (green) - age upon outcome, animal id, animal type, breed, date time, outcome subtype, outcometype, sex, name
#00AAF2 (blue) - filter data, bottom right scroll numbers

#**********************#**********************#**********************#**********************#**********************#**********************


#IMPLEMENT THE CHANGES IN ORDER TO CREATE CORRECT FUNCIONALITY OF THE SEARCH BOX FOR DIFFERENT TYPES OF RESCUE ANIMALS.
 ############# #############  #############  #############  ############# #############

# create the data so the search box can sift through the data of water rescue,
# mountain or wilderness rescue, Disaster or individual tracking

#data = {'type': ['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking', 'Water Rescue'],
        #'location': ['Lake', 'Mountains', 'City', 'River'],
        #'animal': ['Dog', 'Human', 'Cat', 'Dog']}

#df = pd.DataFrame()

#app.layout = html.Div([
    #dcc.Dropdown(
        #id='dropdown',
        #options=[
            ##{'label': 'All', 'value': 'All'},
            #{'label': 'Water Rescue', 'value': 'Water Rescue'},
            #{'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            #{'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
        #],
        #value='All'
    #),
#dash_table.DataTable(
        #id='datatable-id',
        #columns=[{"name": i, "id": i} for i in df.columns],
        #data=df.to_dict('records'),
        #filter_action="native",  # Enable built-in filtering UI
    #),
#])
#**********************#**********************#**********************#**********************#**********************#**********************

db = animalShelter(username, password, host, database, collection, port)
db.username = username
db.password = password
db.host = host
db.database_name = database
db.collection_name = collection
db.port = port
db._connect()




app = Dash(__name__)

image_filename = 'Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('Emergency Animal Rescue Dashboard', style={'color': '#d2d2d2 ','margin-top': '9%'}))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px', 'background-color': '#171a1e'}),
    html.Hr(),
    html.Div([
    dcc.Dropdown(['Water Rescue', 'Mountain or Wilderness Rescue','Disaster or Individual Tracking',
                  ], id='dropdown', style={'color': 'black'}),html.Div(id='dd-output-container'), 
]),
    html.Hr(),
    dash_table.DataTable(
       
        id='datatable-id', 
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable=False,
        row_selectable=False,
        selected_columns=[],
        selected_rows=[],
        page_action='native',
        page_current=0,
        page_size=10,
        
        style_data_conditional=[
        {'if': {'column_id': 'age_upon_outcome'},'color': ' #53e51d', 'background-color': '#242331' },
        {'if': {'column_id': 'animal_id'},'color': ' #53e51d' , 'background-color': '#242331'},
        {'if': {'column_id': 'animal_type'},'color': ' #53e51d' , 'background-color': '#242331'},
        {'if': {'column_id': 'breed'},'color': ' #53e51d' , 'background-color': '#242331'},
        {'if': {'column_id': 'date_time'},'color': '#53e51d' , 'background-color': '#242331'},
        {'if': {'column_id': 'outcome_subtype'},'color': ' #53e51d' , 'background-color': '#242331'},
        {'if': {'column_id': 'outcome_type'},'color': ' #53e51d' , 'background-color': '#242331'},
        {'if': {'column_id': 'sex_upon_outcome'},'color': ' #53e51d' , 'background-color': '#242331'},
        {'if': {'column_id': 'name'},'color': ' #53e51d' , 'background-color': '#242331'},
        {'if': {'column_id': 'datetime'},'color': ' #53e51d' , 'background-color': '#242331'},
        {'if': {'column_id': 'monthyear'},'color': ' #53e51d' , 'background-color': '#242331'},
        {'if':{'column_id': 'date_of_birth'},'color': '#00AAF2' , 'background-color': '#242331'},
        {'if':{'column_id': 'location_lat'} ,'color': '#00AAF2' , 'background-color': '#242331'},
        {'if':{'column_id': 'location_long'} ,'color': '#00AAF2' , 'background-color': '#242331'},
        {'if':{'column_id': 'age_upon_outcome_in_weeks'},'color': '#00AAF2' , 'background-color': '#242331'},
        {'if':{'column_id': 'color'},'color': '#00AAF2' , 'background-color': '#242331'},
        {'if':{'column_id': 'Unnamed: 0'},'color': '#00AAF2' , 'background-color': '#242331'},
        
      
        ],
  
    ),
     html.Hr(),
    html.Br(),
   
    html.Div(className='row' , children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
    html.Br(),
    html.Hr(),
    ])
])


@app.callback(Output('datatable-id', 'data'),
              [Input('dropdown', 'value')])


def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        filtered_df = df[df['type'] == 'Water Rescue']
    elif filter_type == 'Mountain or Wilderness Rescue':
        filtered_df = df[df['type'] == 'Mountain or Wilderness Rescue']
    elif filter_type == 'Disaster or Individual Tracking':
        filtered_df = df[df['type'] == 'Disaster or Individual Tracking']
    else:  # Reset
        filtered_df = df

    return filtered_df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Animals')

    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('datatable-id', 'select_data_conditional' 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    if not index:
        return []

    row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]
    
   
if __name__ == '__main__':
    app.run(debug=True)
    
    
     #To run type: http://127.0.0.1:8050/ in the browser after downloading the proper libraries and code base on your OS.

127.0.0.1 - - [13/Apr/2025 14:18:12] code 400, message Bad request syntax ('+\x01\x00\x00[<î\x7f\x00\x00\x00\x00Ô\x07\x00\x00\x00\x00\x00\x00admin.$cmd\x00\x00\x00\x00\x00ÿÿÿÿ\x04\x01\x00\x00\x10ismaster\x00\x01\x00\x00\x00\x08helloOk\x00\x01\x03client\x00ß\x00\x00\x00\x03driver\x00-\x00\x00\x00\x02name\x00')
127.0.0.1 - - [13/Apr/2025 14:18:22] code 400, message Bad request syntax ('+\x01\x00\x00ú»¼q\x00\x00\x00\x00Ô\x07\x00\x00\x00\x00\x00\x00admin.$cmd\x00\x00\x00\x00\x00ÿÿÿÿ\x04\x01\x00\x00\x10ismaster\x00\x01\x00\x00\x00\x08helloOk\x00\x01\x03client\x00ß\x00\x00\x00\x03driver\x00-\x00\x00\x00\x02name\x00')
127.0.0.1 - - [13/Apr/2025 14:18:32] code 400, message Bad HTTP/0.9 request type ('+\x01\x00\x00lTÙ')
127.0.0.1 - - [13/Apr/2025 14:18:42] code 400, message Bad request syntax ("+\x01\x00\x00Ã\x8d5'\x00\x00\x00\x00Ô\x07\x00\x00\x00\x00\x00\x00admin.$cmd\x00\x00\x00\x00\x00ÿÿÿÿ\x04\x01\x00\x00\x10ismaster\x00\x01\x00\x00\x00\x08helloOk\x00\x01\x03client\x00ß\x00\x00\x00\x03driver